## Query Census Tract Data and Combine with Need Data 
* Author: Callie Clark
* Last Updated: 6/8/2024

In [1]:
MY_API_KEY='insert yours here'

In [1]:
%matplotlib inline
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Set variables
year='2021'
dsource='acs'
dname='acs5'
#cols='GEONAME,NATURALINC,DOMESTICMIG,INTERNATIONALMIG'
state='36'
county='005,047,061,081,085'
# keyfile='census_key.txt'
# outfile='zbp2017.json'

In [13]:
base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}/groups/B01003'#B19019'
base_url

'https://api.census.gov/data/2021/acs/acs5/groups/B01003'

In [14]:
#Retrieve Data
response=requests.get(base_url)
print(response.text)

{
  "variables": {
    "B01003_001E": {
      "label": "Estimate!!Total",
      "concept": "TOTAL POPULATION",
      "predicateType": "long",
      "group": "B01003",
      "limit": 0,
      "predicateOnly": true,
      "universe": "Total population"
    },
    "B01003_001M": {
      "label": "Margin of Error!!Total",
      "concept": "TOTAL POPULATION",
      "predicateType": "long",
      "group": "B01003",
      "limit": 0,
      "predicateOnly": true,
      "universe": "Total population"
    },
    "B01003_001MA": {
      "label": "Annotation of Margin of Error!!Total",
      "concept": "TOTAL POPULATION",
      "predicateType": "string",
      "group": "B01003",
      "limit": 0,
      "predicateOnly": true,
      "universe": "Total population"
    },
    "B01003_001EA": {
      "label": "Annotation of Estimate!!Total",
      "concept": "TOTAL POPULATION",
      "predicateType": "string",
      "group": "B01003",
      "limit": 0,
      "predicateOnly": true,
      "universe": "To

In [19]:
def get_query_url(year, variables):
    # API Reference: https://www.census.gov/data/developers/guidance/api-user-guide.Example_API_Queries.html
    # Data Dictionary: https://api.census.gov/data.html
    host = 'https://api.census.gov/data'
    year = f'/{year}'
    dataset_acronym = '/acs/acs5'
    g = '?get='
    location = '&for=tract:*&in=county:005,047,061,081,085&in=state:36'
    usr_key = "&key=971e994474224126ccfcb72e249537ed4640a48b"

    query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"

    return query_url

In [52]:
query_url=get_query_url(2021, "B19019_001E,B19019_005E,B08201_002E,B08201_001E,B01003_001E,B08301_003E,B08301_010E,B17001_002E,B17001_001E")



In [53]:
response = requests.get(query_url)
print(response.text)
data=response.json()

[["B19019_001E","B19019_005E","B08201_002E","B08201_001E","B01003_001E","B08301_003E","B08301_010E","B17001_002E","B17001_001E","state","county","tract"],
["-666666666","-666666666","0","0","6661","0","0","0","0","36","005","000100"],
["70867","110539","328","1392","4453","766","619","1014","4453","36","005","000200"],
["98090","139479","561","2199","6000","1628","1077","240","6000","36","005","000400"],
["40033","97560","1214","2187","6038","880","1154","951","5826","36","005","001600"],
["55924","77647","615","885","2168","214","584","596","2164","36","005","001901"],
["60804","32212","242","376","1399","65","470","277","1270","36","005","001902"],
["-666666666","-666666666","0","0","0","0","0","0","0","36","005","001903"],
["-666666666","-666666666","0","0","0","0","0","0","0","36","005","001904"],
["20870","16059","1297","1759","4694","222","842","2187","4694","36","005","002001"],
["-666666666","90814","1162","1904","4274","566","954","1480","4184","36","005","002002"],
["22177","

In [54]:
var_col_names={"B19019_001E":"Median_Income","B19019_005E":"Median_Income_4per_hh","B01003_001E":"Total_Population",'B08201_002E':'zero_veh_hh_numer',
               'B08201_001E':'num_veh_hh_denom','B08301_003E':'commute_drive','B08301_010E':'commute_transit','B17001_002E':'poverty','B17001_001E':'poverty_denom'}
               

In [81]:
df=pd.DataFrame(data[1:],columns=data[0])
df['GEOID']=(df['state'].astype('str')+df['county'].astype('str')+df['tract'].astype('str')).astype('int')
df.set_index('GEOID',inplace=True)
df.replace('-666666666',np.nan,inplace=True)
df.rename(columns=var_col_names,inplace=True)
df['percent_car_owners']=round((df['num_veh_hh_denom'].astype('int')-df['zero_veh_hh_numer'].astype('int'))/df['num_veh_hh_denom'].astype('int'),4)
df['percent_commute_drive']=round((df['commute_drive'].astype('int'))/df['Total_Population'].astype('int'),4)
df['percent_commute_transit']=round((df['commute_transit'].astype('int'))/df['Total_Population'].astype('int'),4)
df['percent_poverty']=round((df['poverty'].astype('int'))/df['poverty_denom'].astype('int'),4)

df.drop(columns=['num_veh_hh_denom','zero_veh_hh_numer','county','state','tract'],inplace=True)
df.Total_Population=df.Total_Population.astype('int')
df

,Median_Income,Median_Income_4per_hh,Total_Population,commute_drive,commute_transit,poverty,poverty_denom,percent_car_owners,percent_commute_drive,percent_commute_transit,percent_poverty
GEOID,,,,,,,,,,,
36005000100,NaN,NaN,6661,0,0,0,0,NaN,0.0000,0.0000,NaN
36005000200,70867,110539,4453,766,619,1014,4453,0.7644,0.1720,0.1390,0.2277
36005000400,98090,139479,6000,1628,1077,240,6000,0.7449,0.2713,0.1795,0.0400
36005001600,40033,97560,6038,880,1154,951,5826,0.4449,0.1457,0.1911,0.1632
36005001901,55924,77647,2168,214,584,596,2164,0.3051,0.0987,0.2694,0.2754
...,...,...,...,...,...,...,...,...,...,...,...
36085030302,85842,92011,6663,1947,775,1235,6638,0.8821,0.2922,0.1163,0.1861
36085031901,NaN,NaN,3422,241,568,1229,3386,0.3162,0.0704,0.1660,0.3630
36085031902,76066,175025,5298,984,1015,1235,5275,0.6431,0.1857,0.1916,0.2341


In [59]:
df.describe()

,Total_Population,percent_car_owners,percent_commute_drive,percent_commute_transit,percent_poverty
count,2327.000000,2229.00000,2239.000000,2239.000000,2235.000000
mean,3754.210142,0.51680,0.112677,0.224359,0.162314
std,2089.412670,0.24385,0.080549,0.093447,0.122662
min,0.000000,0.00000,0.000000,0.000000,0.000000
25%,2298.500000,0.31120,0.048700,0.160300,0.073600
50%,3520.000000,0.49720,0.092400,0.218900,0.130500
75%,4919.000000,0.72750,0.162500,0.284400,0.214500
max,16629.000000,1.00000,0.652200,1.000000,1.000000


In [60]:
df['Total_Population'].quantile(0.05)

529.2000000000002

In [27]:
df.dtypes

Median_Income               object
Median_Income_4per_hh       object
Total_Population             int64
commute_drive               object
commute_transit             object
poverty                     object
percent_car_owners         float64
percent_commute_drive      float64
percent_commute_transit    float64
percent_poverty            float64
dtype: object

In [28]:
df.describe()

,Total_Population,percent_car_owners,percent_commute_drive,percent_commute_transit,percent_poverty
count,2327.000000,2229.00000,2239.000000,2239.000000,2239.000000
mean,3754.210142,0.51680,0.112677,0.224359,0.158326
std,2089.412670,0.24385,0.080549,0.093447,0.117354
min,0.000000,0.00000,0.000000,0.000000,0.000000
25%,2298.500000,0.31120,0.048700,0.160300,0.070950
50%,3520.000000,0.49720,0.092400,0.218900,0.128400
75%,4919.000000,0.72750,0.162500,0.284400,0.212300
max,16629.000000,1.00000,0.652200,1.000000,0.771400


In [61]:
import geopandas as gpd
tract_shp = gpd.read_file('geo_files/nyct2020.shp')
nyc_CT=tract_shp[tract_shp.BoroName.isin(['Manhattan', 'Bronx', 'Queens', 'Staten Island', 'Brooklyn'])]         
nyc_CT=nyc_CT.to_crs(4326)
nyc_CT.set_index('GEOID',inplace=True)
nyc_CT.index=nyc_CT.index.astype('int')
nyc_CT.head()

,CTLabel,BoroCode,BoroName,CT2020,BoroCT2020,CDEligibil,NTAName,NTA2020,CDTA2020,CDTANAME,PUMA,Shape_Leng,Shape_Area,geometry
GEOID,,,,,,,,,,,,,,
36061000100,1,1,Manhattan,000100,1000100,None,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),4121,10833.043929,1.843005e+06,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
36061000201,2.01,1,Manhattan,000201,1000201,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),4103,4754.495247,9.723121e+05,"POLYGON ((-73.98450 40.70951, -73.98655 40.709..."
36061000600,6,1,Manhattan,000600,1000600,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),4103,6976.286215,2.582705e+06,"POLYGON ((-73.99022 40.71440, -73.98934 40.714..."
36061001401,14.01,1,Manhattan,001401,1001401,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),4103,5075.332000,1.006117e+06,"POLYGON ((-73.98837 40.71645, -73.98754 40.716..."
36061001402,14.02,1,Manhattan,001402,1001402,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),4103,4459.156019,1.226206e+06,"POLYGON ((-73.98507 40.71908, -73.98423 40.718..."


In [65]:
df=df[df['Total_Population']>529]
df_geo=df.join(nyc_CT)
#df_geo['geometry'] = df_geo['geometry'].apply(wkt.loads)
df_geo=gpd.GeoDataFrame(df_geo, geometry=df_geo.geometry)
df_geo.set_crs(4326,allow_override=True)
df_geo=df_geo[df_geo['Total_Population']>529]
df_geo_filt=df_geo[(df_geo['Median_Income'].isna())&(df_geo['Median_Income_4per_hh'].isna())]#df_geo[(df_geo['Total_Population']>10)&(df_geo['Total_Population']<529)]


In [66]:
df[df['percent_car_owners'].isna()]
#df[df['percent_commute_drive'].isna()]

,Median_Income,Median_Income_4per_hh,Total_Population,commute_drive,commute_transit,poverty,poverty_denom,percent_car_owners,percent_commute_drive,percent_commute_transit,percent_poverty
GEOID,,,,,,,,,,,
36005000100,NaN,NaN,6661,0,0,0,0,NaN,0.0000,0.0000,NaN
36005009302,NaN,NaN,566,0,0,0,0,NaN,0.0000,0.0000,NaN
36005028400,NaN,NaN,544,0,0,86,130,NaN,0.0000,0.0000,0.6615
36047001801,NaN,NaN,2021,0,0,0,0,NaN,0.0000,0.0000,NaN
36061024000,NaN,NaN,1845,4,267,1157,1493,NaN,0.0022,0.1447,0.7749


In [82]:
#df_final=df[df['Total_Population']>529] #implement a 500 person minimum cutoff
df_final=df.dropna(subset=['Median_Income'],how='all')
df_final.Median_Income=df_final.Median_Income.astype('int')
#df_final[df_final['percent_car_owners'].isna()]
df_final#[df_final['percent_commute_drive'].isna()]

/tmp/ipykernel_50619/904346934.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.Median_Income=df_final.Median_Income.astype('int')


,Median_Income,Median_Income_4per_hh,Total_Population,commute_drive,commute_transit,poverty,poverty_denom,percent_car_owners,percent_commute_drive,percent_commute_transit,percent_poverty
GEOID,,,,,,,,,,,
36005000200,70867,110539,4453,766,619,1014,4453,0.7644,0.1720,0.1390,0.2277
36005000400,98090,139479,6000,1628,1077,240,6000,0.7449,0.2713,0.1795,0.0400
36005001600,40033,97560,6038,880,1154,951,5826,0.4449,0.1457,0.1911,0.1632
36005001901,55924,77647,2168,214,584,596,2164,0.3051,0.0987,0.2694,0.2754
36005001902,60804,32212,1399,65,470,277,1270,0.3564,0.0465,0.3360,0.2181
...,...,...,...,...,...,...,...,...,...,...,...
36085029106,127671,184135,3529,1158,223,217,3529,0.9638,0.3281,0.0632,0.0615
36085030301,95913,143026,5628,1656,747,171,5621,0.8888,0.2942,0.1327,0.0304
36085030302,85842,92011,6663,1947,775,1235,6638,0.8821,0.2922,0.1163,0.1861


In [83]:
df_final.join(nyc_CT,how='left')[['BoroName','Median_Income','Median_Income_4per_hh','percent_car_owners','percent_poverty']]

,BoroName,Median_Income,Median_Income_4per_hh,percent_car_owners,percent_poverty
GEOID,,,,,
36005000200,Bronx,70867,110539,0.7644,0.2277
36005000400,Bronx,98090,139479,0.7449,0.0400
36005001600,Bronx,40033,97560,0.4449,0.1632
36005001901,Bronx,55924,77647,0.3051,0.2754
36005001902,Bronx,60804,32212,0.3564,0.2181
...,...,...,...,...,...
36085029106,Staten Island,127671,184135,0.9638,0.0615
36085030301,Staten Island,95913,143026,0.8888,0.0304
36085030302,Staten Island,85842,92011,0.8821,0.1861


In [86]:

avg_vals=df_final.join(nyc_CT,how='left')[['BoroName','Median_Income','Median_Income_4per_hh','percent_car_owners','percent_poverty']].dropna()
avg_vals['Median_Income_4per_hh']=avg_vals['Median_Income_4per_hh'].astype('int')

avg_vals.groupby('BoroName').mean()

,Median_Income,Median_Income_4per_hh,percent_car_owners,percent_poverty
BoroName,,,,
Bronx,52560.825095,80325.562738,0.449532,0.241329
Brooklyn,79761.843478,123657.320000,0.497795,0.161026
Manhattan,113721.607656,178510.483254,0.242789,0.143021
Queens,82175.050186,115220.237918,0.677790,0.106818
Staten Island,94763.871287,142111.653465,0.858042,0.099639


In [87]:
df_final[df_final['Median_Income_4per_hh'].isna()]

,Median_Income,Median_Income_4per_hh,Total_Population,commute_drive,commute_transit,poverty,poverty_denom,percent_car_owners,percent_commute_drive,percent_commute_transit,percent_poverty
GEOID,,,,,,,,,,,
36005002500,25954,NaN,5662,117,1220,2557,5642,0.1334,0.0207,0.2155,0.4532
36005002800,54321,NaN,5372,979,1121,153,5338,0.6850,0.1822,0.2087,0.0287
36005003700,37321,NaN,277,27,44,80,262,0.1203,0.0975,0.1588,0.3053
36005003900,20049,NaN,6710,172,1586,2656,6671,0.2761,0.0256,0.2364,0.3981
36005004001,64087,NaN,1574,238,274,303,1574,0.7324,0.1512,0.1741,0.1925
...,...,...,...,...,...,...,...,...,...,...,...
36085020701,63750,NaN,3168,555,474,619,3149,0.7910,0.1752,0.1496,0.1966
36085020702,28523,NaN,2311,348,204,951,2311,0.4888,0.1506,0.0883,0.4115
36085024401,118592,NaN,5704,1553,355,235,5704,0.8971,0.2723,0.0622,0.0412


In [88]:
income_corr=df_final.join(nyc_CT,how='left')[['Median_Income','Median_Income_4per_hh','BoroName']].dropna()
income_corr['Median_Income_4per_hh']=income_corr['Median_Income_4per_hh'].astype('int')
income_corr['Median_Income']=income_corr['Median_Income'].astype('int')


income_corr_df=income_corr.groupby('BoroName').mean()
income_corr_df['Ratio']=income_corr_df['Median_Income_4per_hh']/income_corr_df['Median_Income']
income_corr_df

,Median_Income,Median_Income_4per_hh,Ratio
BoroName,,,
Bronx,52560.825095,80325.562738,1.528240
Brooklyn,79761.843478,123657.320000,1.550332
Manhattan,113721.607656,178510.483254,1.569715
Queens,82175.050186,115220.237918,1.402132
Staten Island,94763.871287,142111.653465,1.499640


In [89]:
import scipy.stats #evaluate if correlation is higher globally or by borough 
df_=df_final.join(nyc_CT,how='left')[['Median_Income','Median_Income_4per_hh','BoroName']].dropna()
df_.Median_Income_4per_hh=df_.Median_Income_4per_hh.astype('int')
for i in df_.BoroName.unique():
    print(i)
    df_i=df_[df_.BoroName==i]  
    corr,p=scipy.stats.pearsonr(df_i['Median_Income'],df_i['Median_Income_4per_hh'])
    print(corr,p)

Bronx
0.781100518067288 2.6075089092811942e-55
Brooklyn
0.7389758461700131 2.379792267438967e-100
Manhattan
0.775370294908356 3.394794965912049e-43
Queens
0.6021580604674808 2.1852652140146865e-54
Staten Island
0.49390212440781517 1.5328159807260645e-07


In [90]:
import scipy.stats
corr,p=scipy.stats.pearsonr(income_corr['Median_Income'],income_corr['Median_Income_4per_hh'])
print(corr,p)

0.7602110909473119 9.754387e-318


In [91]:
income_corr=df_final.join(nyc_CT,how='left')[['Median_Income','Median_Income_4per_hh','BoroName']].dropna()
income_corr['Median_Income_4per_hh']=income_corr['Median_Income_4per_hh'].astype('int')
income_corr['Median_Income']=income_corr['Median_Income'].astype('int')



income_corr['Ratio']=income_corr['Median_Income_4per_hh']/income_corr['Median_Income']
income_corr

,Median_Income,Median_Income_4per_hh,BoroName,Ratio
GEOID,,,,
36005000200,70867,110539,Bronx,1.559809
36005000400,98090,139479,Bronx,1.421949
36005001600,40033,97560,Bronx,2.436989
36005001901,55924,77647,Bronx,1.388438
36005001902,60804,32212,Bronx,0.529768
...,...,...,...,...
36085029106,127671,184135,Staten Island,1.442262
36085030301,95913,143026,Staten Island,1.491206
36085030302,85842,92011,Staten Island,1.071865


In [92]:
income_corr.Ratio.mean() #how we determined the coeff 

1.5535578644378167

In [93]:
df_final.loc[df_final.Median_Income_4per_hh.isna(),'Median_Income_4per_hh']=df_final.loc[df_final.Median_Income_4per_hh.isna(),'Median_Income'].astype('int')*1.55


In [95]:
df_final#[df_final.Median_Income_4per_hh.isna()]

,Median_Income,Median_Income_4per_hh,Total_Population,commute_drive,commute_transit,poverty,poverty_denom,percent_car_owners,percent_commute_drive,percent_commute_transit,percent_poverty
GEOID,,,,,,,,,,,
36005000200,70867,110539,4453,766,619,1014,4453,0.7644,0.1720,0.1390,0.2277
36005000400,98090,139479,6000,1628,1077,240,6000,0.7449,0.2713,0.1795,0.0400
36005001600,40033,97560,6038,880,1154,951,5826,0.4449,0.1457,0.1911,0.1632
36005001901,55924,77647,2168,214,584,596,2164,0.3051,0.0987,0.2694,0.2754
36005001902,60804,32212,1399,65,470,277,1270,0.3564,0.0465,0.3360,0.2181
...,...,...,...,...,...,...,...,...,...,...,...
36085029106,127671,184135,3529,1158,223,217,3529,0.9638,0.3281,0.0632,0.0615
36085030301,95913,143026,5628,1656,747,171,5621,0.8888,0.2942,0.1327,0.0304
36085030302,85842,92011,6663,1947,775,1235,6638,0.8821,0.2922,0.1163,0.1861


In [96]:
df_final[['percent_commute_drive','percent_car_owners','Median_Income','percent_commute_transit']].corr(method='pearson').round(2)

,percent_commute_drive,percent_car_owners,Median_Income,percent_commute_transit
percent_commute_drive,1.00,0.85,0.12,-0.54
percent_car_owners,0.85,1.00,0.21,-0.51
Median_Income,0.12,0.21,1.00,0.07
percent_commute_transit,-0.54,-0.51,0.07,1.00


In [97]:
avg_vals=df_final.join(nyc_CT,how='left')[['BoroName','Median_Income','Median_Income_4per_hh','percent_car_owners','percent_poverty']].dropna()
avg_vals['Median_Income_4per_hh']=avg_vals['Median_Income_4per_hh'].astype('int')

avg_vals.groupby('BoroName').mean()

,Median_Income,Median_Income_4per_hh,percent_car_owners,percent_poverty
BoroName,,,,
Bronx,49992.766962,76601.368732,0.432693,0.252266
Brooklyn,75850.006494,117587.158442,0.484551,0.174307
Manhattan,108779.198653,170185.323232,0.226230,0.148579
Queens,80915.424963,115705.164933,0.667630,0.110876
Staten Island,92110.299145,138651.188034,0.835967,0.110705


In [100]:
df_final[['Median_Income', 'Median_Income_4per_hh', 'Total_Population',
       'percent_car_owners','percent_poverty']]

,Median_Income,Median_Income_4per_hh,Total_Population,percent_car_owners,percent_poverty
GEOID,,,,,
36005000200,70867,110539,4453,0.7644,0.2277
36005000400,98090,139479,6000,0.7449,0.0400
36005001600,40033,97560,6038,0.4449,0.1632
36005001901,55924,77647,2168,0.3051,0.2754
36005001902,60804,32212,1399,0.3564,0.2181
...,...,...,...,...,...
36085029106,127671,184135,3529,0.9638,0.0615
36085030301,95913,143026,5628,0.8888,0.0304
36085030302,85842,92011,6663,0.8821,0.1861


In [101]:
df_final[['Median_Income', 'Median_Income_4per_hh', 'Total_Population',
       'percent_car_owners','percent_poverty']].to_csv('ACS_data_5y_2021.csv')

In [109]:
  

#Pull in Food Insecurity Data or other need data here if desired

df_demo_FI=df_final[['Median_Income', 'Median_Income_4per_hh', 'Total_Population',
       'percent_car_owners','percent_poverty']]



tcl_data=pd.read_csv('true_cost_of_living_data.csv',index_col=0)
df_demo_FI_tcl=df_demo_FI.join(tcl_data, how='left')
df_demo_FI_tcl
df_demo_FI_tcl=df_demo_FI_tcl.drop_duplicates()
df_demo_FI_tcl['Income_minus_TCOL']=(df_demo_FI_tcl['Median_Income_4per_hh'])-df_demo_FI_tcl['TCoL_2023'] #use 2023 column (based on 2021 ACS)
df_demo_FI_tcl['below_TCOL']=(df_demo_FI_tcl['Income_minus_TCOL']<0)*1
df_demo_FI_tcl.drop(columns=['TCoL_2021'],inplace=True)
df_demo_FI_tcl
df_demo_FI_tcl.to_csv('CT_demog_FI_TCL.csv')
